In [1]:
import requests
from bs4 import BeautifulSoup
import pandas  as  pd

In [2]:
url = "https://repositorio.lamolina.edu.pe/recent-submissions?offset="

# Realizar una petición HTTP a la URL
response = requests.get(url)

# Analizar el contenido HTML de la página usando Beautiful Soup
soup = BeautifulSoup(response.content, "html.parser")

# Crea el limite superior del for tomando en cuenta la cantidad de tesis expuestas en  la pagina
limsup=int((soup.find("p",{"class":"pagination-info"})).text[-4:])/20+1

In [3]:
hrefs=[]
# Iterar sobre todos los valores de i desde 0 hasta 100
for i in range(0, int(limsup)):
    # Crear la URL de la página a scrappear
    url = "https://repositorio.lamolina.edu.pe/recent-submissions?offset=" + str(i*20)

    # Realizar una petición HTTP a la URL
    response = requests.get(url)

    # Analizar el contenido HTML de la página usando Beautiful Soup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extraer todos los enlaces de la página
    links = soup.find_all("h4",{"class":"artifact-title"})


    # Iterar sobre todos los enlaces extraídos y mostrarlos por pantalla
    for link in links:
        hrefs.append("https://repositorio.lamolina.edu.pe"+link.find("a").get("href")+"?show=full")

In [4]:
# Se declaran las listas donde se almacenaran los datos
institucion=[]
titulo=[]
autor=[]
grado=[]
asesor=[]
resumen=[]
anio=[]
vacio=""

In [5]:
for i in range(len(hrefs)):
    response=requests.get(hrefs[i])
    
    # Analizar el contenido HTML de la página usando Beautiful Soup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extraer la tabla que deseas
    table = soup.find("table", {"class": "ds-includeSet-table detailtable table table-striped table-hover"})

    # Convertir la tabla en un DataFrame de Pandas
    df = pd.read_html(str(table),index_col=0)[0]
    
    # Nos quedamos con la parte de la tabla que nos importa
    df=df[:][1]
    
    # Scrappeo de datos
    institucion.append(df.get("dc.publisher", vacio))   
    titulo.append(df.get("dc.title", vacio))
    grado.append(df.get("dc.type", vacio))
    anio.append(df.get("dc.date.issued", vacio))

    if "dc.contributor.author" in df:
        if type(df["dc.contributor.author"])==float:
            autor.append(vacio)
        else:
            if isinstance(df["dc.contributor.author"], str) :
                autor.append(df["dc.contributor.author"])
            else :
                autor.append(df["dc.contributor.author"][:].apply(lambda x: x).tolist())     
    else:
        autor.append(vacio)
        
    if "dc.contributor.advisor" in df:
        if type(df["dc.contributor.advisor"])==float :
            asesor.append(vacio)
        else:
            if isinstance(df["dc.contributor.advisor"], str) :
                asesor.append(df["dc.contributor.advisor"]) 
            else :
                asesor.append(df["dc.contributor.advisor"].apply(lambda x: x).tolist())
    else:
        asesor.append(vacio)
    
    if "dc.description.abstract" in df:
        if type(df["dc.description.abstract"])==float :
            resumen.append(vacio)
        else:
            if isinstance(df["dc.description.abstract"], str):
                resumen.append(df["dc.description.abstract"])
            else:
                resumen.append(df["dc.description.abstract"][0])
    else:
        resumen.append(vacio)

In [6]:
datosexportar = pd.DataFrame(list(zip(anio,institucion,titulo,autor,grado,asesor,resumen)),columns =["Año","Institución","Titulo","Autor","Grado","Asesor","Resumen"])

In [7]:
datosexportar.to_csv("repositorio_agraria_tesis.csv", sep=',')

In [8]:
datosexportar

,Año,Institución,Titulo,Autor,Grado,Asesor,Resumen
0,2022,Universidad Nacional Agraria La Molina,Predicción del siniestro de vehículos particul...,"Ramirez Navarro, Vanessa Judith",info:eu-repo/semantics/bachelorThesis,"Chue Gallardo, Jorge",Tras el incremento de siniestros vehiculares d...
1,2022,Universidad Nacional Agraria La Molina,Fuentes de magnesio y silicio en el rendimient...,"Lucero Rafael, Victor Antonio de Jesus",info:eu-repo/semantics/bachelorThesis,"Casas Díaz, Andrés Virgilio",El maíz amarillo duro (Zea mays L.) es un cult...
2,2022,Universidad Nacional Agraria La Molina,Manejo integrado de plagas en el cultivo de ar...,"Amézquita Zegarra, Giovanni",info:eu-repo/semantics/bachelorThesis,"Rodríguez Soto, Gilberto",Ante el crecimiento notorio que ha tenido el c...
3,2022,Universidad Nacional Agraria La Molina,Programa Vaso de Leche y ración alimenticia pa...,"Sedano David, Virginia Consuelo",info:eu-repo/semantics/bachelorThesis,"García Torres, Silvia Melissa",El Programa del Vaso de Leche creado por ley N...
4,2022,Universidad Nacional Agraria La Molina,Desarrollo de una leche semidescremada sin lac...,"Peve Gonzales, Omar Abel",info:eu-repo/semantics/bachelorThesis,"Elías Peñafiel, Carlos Cesar Augusto",El objetivo de este trabajo fue desarrollar un...
...,...,...,...,...,...,...,...
4103,2007,Universidad Nacional Agraria La Molina,"Preservación con sales CCB y CCA-C, durabilida...","Guerrero Ruíz, William Omar",info:eu-repo/semantics/bachelorThesis,"Trujillo Cuéllar, Florencio Teodoro",La motivación de este trabajo de tesis ha sido...
4104,2007,Universidad Nacional Agraria La Molina,Influencia de la densidad de la madera en la c...,"Santillán Saldaña, Tatiana Silvana",info:eu-repo/semantics/bachelorThesis,"Canchucaja Rojas, Julio César",Las industrias de tableros de partículas son u...
4105,2007,Universidad Nacional Agraria La Molina,Caracterización de las propiedades tintóreas d...,"Masias Brocker, Katia",info:eu-repo/semantics/bachelorThesis,"Gonzáles Mora, Héctor Enrique",En los últimos años se observa un incremento e...
4106,2006,Universidad Nacional Agraria La Molina,"Propiedades físico-mecánicas del Aliso, Alnus ...","López Fierro, Jorge Eduardo",info:eu-repo/semantics/bachelorThesis,"Acevedo Mallque, Moisés Pascual","El distrito de Chalaco, ubicado en la provinci..."
